In [1]:
import pandas as pd

In [3]:
app_att = pd.read_csv("../data_source/apartment_attributes.csv")
user_viewings = pd.read_csv("../data_source/user_viewings.csv")
app = pd.read_csv("../data_source/apartments.csv")

In [4]:
# Merge the dataframes
apartments = app_att.merge(app, on="id")

In [7]:
apartments.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 24 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   id                       10000 non-null  int64  
 1   category                 10000 non-null  object 
 2   body                     10000 non-null  object 
 3   amenities                10000 non-null  object 
 4   bathrooms                10000 non-null  float64
 5   bedrooms                 10000 non-null  float64
 6   fee                      0 non-null      float64
 7   has_photo                10000 non-null  object 
 8   pets_allowed             5837 non-null   object 
 9   price_display            10000 non-null  object 
 10  price_type               10000 non-null  object 
 11  square_feet              10000 non-null  int64  
 12  address                  10000 non-null  object 
 13  cityname                 9923 non-null   object 
 14  state                  

In [8]:
apartments.isnull().sum()

id                             0
category                       0
body                           0
amenities                      0
bathrooms                      0
bedrooms                       0
fee                        10000
has_photo                      0
pets_allowed                4163
price_display                  0
price_type                     0
square_feet                    0
address                        0
cityname                      77
state                         77
latitude                      10
longitude                     10
title                          0
source                         0
price                          0
currency                       0
listing_created_on             0
is_active                      0
last_modified_timestamp        0
dtype: int64

In [10]:
# Average Listings Price (Daily)
apartments['listing_created_on'] = pd.to_datetime(apartments['listing_created_on'])
average_listing_price = apartments.groupby(apartments['listing_created_on'].dt.date)['price'].mean().reset_index(name='average_price')  
average_listing_price

,listing_created_on,average_price
0,2024-05-16,1520.200595
1,2024-05-17,1473.570505
2,2024-05-18,1464.841981


In [17]:
# Daily viewings count

# Convert viewed_at to datetime and then calculate daily viewings count
user_viewings['viewed_at'] = pd.to_datetime(user_viewings['viewed_at'])
daily_bookings = user_viewings.groupby(user_viewings['viewed_at'].dt.date).size().reset_index(name='total_bookings')

# Daily active listings count
active_listings = apartments[apartments['is_active'] == 1]
active_listings['listing_created_on'] = pd.to_datetime(active_listings['listing_created_on']).dt.date
daily_active_listings = active_listings.groupby('listing_created_on').size().reset_index(name='active_listings')

# Join and compute occupancy rate
occupancy_rate = daily_bookings.merge(daily_active_listings, left_on='viewed_at', right_on='listing_created_on', how='left')
occupancy_rate['occupancy_rate'] = (occupancy_rate['total_bookings'] / occupancy_rate['active_listings']) * 100
occupancy_rate

C:\Users\HeskeyAmoakoFordjour\AppData\Local\Temp\ipykernel_31420\2270174613.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  active_listings['listing_created_on'] = pd.to_datetime(active_listings['listing_created_on']).dt.date


,viewed_at,total_bookings,listing_created_on,active_listings,occupancy_rate
0,2024-05-16,75,2024-05-16,2789.0,2.689136
1,2024-05-17,250,2024-05-17,2672.0,9.356287
2,2024-05-18,370,2024-05-18,2843.0,13.014421
3,2024-05-19,464,NaN,NaN,NaN
4,2024-05-20,463,NaN,NaN,NaN
5,2024-05-21,427,NaN,NaN,NaN
6,2024-05-22,475,NaN,NaN,NaN
7,2024-05-23,409,NaN,NaN,NaN
8,2024-05-24,423,NaN,NaN,NaN
9,2024-05-25,477,NaN,NaN,NaN
